# 09. 한글 텍스트 처리 - 네이버 영화 평점 감성 분석

p530(549)

<br>

## 9.1 한글 NLP 처리의 어려움

- 일반적으로 한글 언어 처리는 영어 등의 라틴어 처리보다 어려움
- 그 주된 원인은 **띄어쓰기**와 **다양한 조사** 때문이다.

### 9.1.1 띄어쓰기

- 띄어쓰기를 잘못했을 때의 의미 왜곡
  - '아버지가 방에 들어가신다.' vs '아버지 가방에 들어가신다'
- 영어는 띄어쓰기를 잘못하면 의미가 왜곡되는 게 아니라 잘못된 또는 없는 단어로 인식된다.

<br>

### 9.1.2 조사

- 조사는 주어나 목적어를 위해 추가됨
- 워낙 경우의 수가 많기 때문에 어근 추출(Stemming/Lemmatization) 등의 전처리 시 제거하기가 까다롭다.
- "집" 이라는 어근 단어를 기준으로  집은, 집이, 집으로, 집에서, 집에 등 다양한 형태의 조사가 존재  
- 또한 "너희 집은 어디 있니?"에서 "집은"의 "은"이 뜻하는 것이 조사인 지 아니면 은(銀)인 지 구분하기 어려움
- "집은"을 "집 은"으로 띄어쓰기를 잘못하면 더 그렇다.

<br>

## 9.2 KoNLPy 소개

- KoNLPy는 파이썬의 대표적인 한글 형태소 패키지이다.
- **형태소**의 사전적 의미 : "단어로서 의미를 가지는 최소 단위"
- **형태소 분석(Morphological analysis)**
  - 말뭉치를 이러한 형태소 어근 단위로 쪼개고 각 형태소에 품사 태깅(POS tagging)을 부착하는 작업을 일반적으로 지칭

- KoNLPy는 기존의 C/C++, Java로 잘 만들어진 한글 형태소 엔진을 파이썬 래퍼(Wrapper) 기반으로 재작성한 패키지이다.
- 기존의 엔진은 그대로 유지한 채 파이썬 기반에서 인터페이스를 제공하기 때문에 검증된 패키지 안정성을 유지할 수 있다.
- 아래의 5개 형태소 분석 모듈을 KoNLPy에서 모두 사용할 수 있다.
  - 꼬꼬마(Kkma)
  - 한나눔(Hannanum)
  - Komoran
  - 은전한닢 프로젝트(Mecab)
  - Twitter
- 뛰어난 형태소 분석으로 인정받고 있는 Mecab의 경우 윈도우 환경에서는 구동되지 않음
- Mecab을 사용하고자 한다면 현재까지는 리눅스 환경의 KoNLPy에서만 가능하다.

<br>

### 9.2.1 KoNLPy 설치

- [공식 설치 문서](https://konlpy-ko.readthedocs.io/ko/v0.4.3/install)를 참조
- 특히 윈도우 운영체제에서 `JPypel` 모듈이 제대로 설치가 되지 않는 경우가 많이 발생

**공식 문서대로 설치가 안될 때 아래와 같은 방법을 통해 설치**

- KoNLPy는 파이썬으로 기존 형태소 분석 엔진을 래퍼한 것이기 때문에 **Java**가 먼저 설치돼 있어야 한다.
- 또한 파이썬에서 Java 클래스를 호출학 ㅣ위한 별도의 모듈인 **`JPype1`**도 함께 필요

**1. conda 명령어를 통해 `jpype1` 설치**

```
conda install -c conda-forge jpype1
```

**2. Java 1.7 버전 이상을 설치 후 JAVA_HOME 설정**
  - 일반적인 JAVA_HOME은 JDK 폴더를 JAVA_HOME으로 설정  
  (C:\Program Files\Java\jdk1.8.0_231)
  - 하지만 KoNLPy의 경우 압축이 풀리는 기준 JDK 폴더가 아니라 `jvm.dll`이 들어 있는 폴더를 JAVA_HOME으로 설정해줘야 한다.  
  (C:\Program Files\Java\jdk1.8.0_231\jre\bin\server)
  - 윈도우의 환경변수에 JAVA_HOME을 C:\Program Files\Java\jdk1.8.0_231\jre\bin\server 으로 설정

**3. `JPype1`과 Java 환경 세팅이 완료됐으면 `pip`를 이용해 KoNLPy를 설치**

```
pip install konlpy
```

<br>

## 9.3 데이터 로딩

- 네이버 영화 평점 데이터는 [깃허브 링크](https://github.com/e9t/nsmc)에서 내려받을 수 있다.
- 전체 데이터 세트(ratings.txt), 학습 데이터 세트(ratings_train.txt), 테스트 데이터 세트(ratings_test.txt)를 모두 내려 받는다.

<br>

### 9.3.1 데이터프레임 생성

- ratings_train.txt 파일을 `DataFrame`으로 로딩하고 데이터를 확인
- ratings_train.txt 파일은 탭(\t)으로 컬럼이 분리돼 있으므로 `read_csv()`의 `sep` 파라미터를 `'\t'`로 설정해 `DataFrame`으로 생성

In [1]:
import pandas as pd

train_df = pd.read_csv('./data/NaverSentimentMovieCorpus/ratings_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


- 학습 데이터 세트의 0과 1의 Label 값 비율 확인
  - 1 : 긍정 감성
  - 0 : 부정 감성

In [2]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

- 0과 1의 비율이 어느 한쪽으로 치우치지 않고 균등한 분포를 나타내고 있음

<br>

### 9.3.2 결측값 처리

- `train_df`의 경우 리뷰 텍스트를 가지는 `document` 컬럼에 `Null`이 일부 존재하므로 이 값은 공백으로 변환
- 문자가 아닌 숫자의 경우 단어적인 의미로 부족하므로 파이썬 정규 표현식 모듈인 `re`를 이용해 이 역시 공백으로 변환
- 테스트 데이터 세트의 경우도 파일을 로딩하고 동일한 데이터 가공을 수행

In [ ]:
import re

train_df = train_df.fillna(' ')

# 정규 표현식을 이용해 숫자를 공백으로 변경 (정규 표현식으로 '\d' 는 숫자를 의미함)
train_df['document'] = train_df['document'].apply(lambda x: re.sub(r"\d+", " ", x))

# 테스트 데이터 세트를 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('./data/NaverSentimentMovieCorpus/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r"\d+", " ", x))